# 各个脚本案例

In [ ]:
!python script/video_to_images.py --video-path ./dataset/video/city.gif

In [ ]:
!python script/edge_smooth.py --dataset shinkai --image-size 256

In [74]:
!python inference_image.py --checkpoint checkpoint/generator_shinkai.pth --src sample/1.jpg --dest 1.png

Weight loaded, ready to predict
Anime image saved to 1.png


In [7]:
!python inference_image.py --checkpoint checkpoint/generator_shinkai_train_24_finetune_5.pth --src sample/mountain.jpg --dest 1.png

Weight loaded, ready to predict
Anime image saved to 1.png


In [5]:
!python inference_image.py --checkpoint checkpoint/freeze_d_face/generator_shinkai_train_24_finetune_10.pth --src sample/city.jpg --dest 1.png

Weight loaded, ready to predict
Anime image saved to 1.png


In [70]:
!python inference_video.py --checkpoint checkpoint/generator_shinkai_24.pth --src dataset/video/city.gif --dest dataset/video/test/1.mp4

Weight loaded, ready to predict
288
160
Animation video saved to dataset/video/test/1.mp4


# 查看checkpoint训练批数

In [88]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='shinkai')
parser.add_argument('--device', type=str, default='cpu')
args = parser.parse_args(args=[])

In [89]:
from utils.common import load_checkpoint,load_weight
from modeling.anime_ganv2 import Generator
from modeling.anime_ganv2 import Discriminator
import os
def load_checkpoint(model, checkpoint_dir, posfix=''):
    path = os.path.join(checkpoint_dir, f'{model.name}{posfix}.pth')
    return load_weight(model, path)
G = Generator('shinkai')
D = Discriminator(args).to(args.device)

start_e = load_checkpoint(G, 'checkpoint')
test = load_checkpoint(D, 'checkpoint')
start_e

24

# kaggle

In [ ]:
!git clone https://github.com/lyh792127928/pytorch-animegan.git

In [ ]:
!python pytorch-animegan/train.py --device 'cpu' --epochs 20 --save_interval 1 --checkpoint_dir pytorch-animegan/checkpoint --train_photo_path train_photo --data-dir pytorch-animegan/dataset --dataset shinkai --batch 4 --init_epochs 4 --use_sn 

In [ ]:
import os
import zipfile
import datetime

def file2zip(packagePath, zipPath):
    '''
  :param packagePath: 文件夹路径
  :param zipPath: 压缩包路径
  :return:
  '''
    zip = zipfile.ZipFile(zipPath, 'w', zipfile.ZIP_DEFLATED)
    for path, dirNames, fileNames in os.walk(packagePath):
        fpath = path.replace(packagePath, '')
        for name in fileNames:
            fullName = os.path.join(path, name)
            name = fpath + '\\' + name
            zip.write(fullName, name)
    zip.close()


if __name__ == "__main__":
    # 文件夹路径
    packagePath = '/kaggle/working/pytorch-animegan/checkpoint'
    zipPath = '/kaggle/working/output.zip'
    if os.path.exists(zipPath):
        os.remove(zipPath)
    file2zip(packagePath, zipPath)
    print("打包完成")
    print(datetime.datetime.utcnow())

# 移动文件，用于图片批量移动

In [ ]:
import os
import shutil
def file(p):
    p=p
    z=os.listdir(p)
    for i in z:
        
        if not (os.path.isfile(os.path.join(p,i))):
            pp=os.path.join(p,i)
            file(pp)
        else:
            zz=os.listdir(p)
            for k in zz:
                paa.append(os.path.join(p,k))
if __name__ == '__main__':
    p = 'dataset/human_face/origin'
    n_p = 'dataset/human_face/style'
    paa = []
    file(p)
    
    for f in paa:
        try:
            shutil.move(f,n_p)
        except:
            continue
    print('移动完成')

In [24]:
x = torch.randn(1,3,256,256)
#x.size()[1:]
torch.tensor(x.size()[1:]).tolist()

[3, 256, 256]

In [22]:
from IPython.display import Markdown, display
onnx_path = 'checkpoint/animegan.onnx'
model_path  = 'checkpoint/animegan.xml'

In [25]:
# Construct the command for Model Optimizer
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --data_type FP16
                 --output_dir "checkpoint/openvino"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "checkpoint/animegan.onnx" --data_type FP16 --output_dir "checkpoint/openvino"`

In [1]:
from utils.common import load_weight
from modeling.anime_ganv2 import Generator
import torch
torch_model = Generator()
load_weight(torch_model,'checkpoint/generator_shinkai.pth')

24

In [ ]:
output_onnx = 'checkpoint/animegan.onnx'
print("==> Exporting model to ONNX format at '{}'".format(output_onnx))
input_names = ["input0"]
output_names = ["output0"]
# 转ONNX 动态输入转换代码
dynamic_axes= {'input0':[0, 2, 3], 'output0':[0,2,3]} #数字0，1等是指张量的维度，表示哪个维度需要动态输入
inputs = torch.randn(1, 3, 256, 256)
onnx_model = torch.onnx.export(torch_model, inputs, output_onnx,input_names=input_names, output_names=output_names,opset_version=11, dynamic_axes=dynamic_axes)

In [1]:
def denormalize_input(images, dtype=None):
    '''
    [-1, 1] -> [0, 255]
    '''
    images = images * 127.5 + 127.5

    if dtype is not None:
        if isinstance(images, torch.Tensor):
            images = images.type(dtype)
        else:
            # numpy.ndarray
            images = images.astype(dtype)

    return images
def normalize_input(images):
    '''
    [0, 255] -> [-1, 1]
    '''
    return images / 127.5 - 1.0

In [3]:
def preprocess_images(images):
    images = images.astype(np.float32)

    # Normalize to [-1, 1]
    images = normalize_input(images)
    #images = torch.from_numpy(images)
    # Add batch dim
    if len(images.shape) == 3:
        images = np.expand_dims(images, 0)
    # channel first
    images = images.transpose(0, 3, 1, 2)

    return images

In [20]:
#openvino
from openvino.runtime import Core, Layout,Type
from openvino.preprocess import PrePostProcessor, ResizeAlgorithm
import numpy as np
import cv2
import time
import torch
 
core = Core()
model=r"checkpoint\animegan.onnx"
model = core.compile_model(model=model)
image = cv2.imread('sample/1.jpg')[:,:,::-1]
#input_tensor = np.expand_dims(image, 0).transpose(0,3,1,2)
input_tensor = preprocess_images(image)
results = model.infer_new_request({0: input_tensor})
predictions = next(iter(results.values()))
pred = denormalize_input(predictions.transpose(0, 2, 3, 1)[0],dtype=np.int16)

In [21]:
pred.shape

(412, 432, 3)

In [22]:
cv2.imshow('test',pred[:,:,::-1])
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
cv2.destroyAllWindows()

In [24]:
cv2.imwrite('2.png',pred[:,:,::-1])

True